### Dependencies


In [34]:
# Data collection, cleaning, storage
import pandas as pd
import numpy as np
np.bool = np.bool_ # https://stackoverflow.com/questions/74893742/how-to-solve-attributeerror-module-numpy-has-no-attribute-bool
# import psycopg2
# from dotenv import load_dotenv
# import os
# import yfinance as yf
# import re
# import os


# import matplotlib.pyplot as plt
# from datetime import datetime, timedelta
# import random
# from gluonts.dataset.common import ListDataset
# from gluonts.torch.model.deepar import DeepAREstimator
# from gluonts.evaluation.backtest import make_evaluation_predictions
# from gluonts.evaluation import Evaluator

# from gluonts.dataset.common import ListDataset
# from gluonts.evaluation.backtest import make_evaluation_predictions
# from gluonts.evaluation import Evaluator
# # Import DeepAR last
# from gluonts.torch.model.deepar import DeepAREstimator

# from sklearn.preprocessing import RobustScaler

#  App dependencies
#  Reload modules to avoid functions not found
import importlib

import functions.database
importlib.reload(functions.database)

from functions.database import (
    get_column_name_and_datatype_dictionary, 
    prepare_sql_queries_and_values,
    insert_data_into_sql_data_base,
    retrieve_data_from_sql,
    add_metadata_columns
)

import functions.machinelearning
importlib.reload(functions.machinelearning)

from functions.machinelearning import (
    prep_data_for_deep_ar_model,
    create_model_and_train,
    generate_forecasts,
    inverse_transform_forecasts,
    get_forecast_data_frames,

)

import functions.ml_evaluation
importlib.reload(functions.ml_evaluation)

from functions.ml_evaluation import (
    get_evaluation_metrics, 
    get_combined_rmse,
    get_experiment_number,
    get_hyperparameters,
)

import functions.applicant_assessment_results
importlib.reload(functions.applicant_assessment_results)

from functions.applicant_assessment_results import (
    assess_affordability,
    get_overall_assessment,
)

import warnings
warnings.filterwarnings('ignore')



# /Users/admin/PycharmProjects/cwb-ml-api/venv/lib/python3.12/site-packages/gluonts/json.py:102: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
#   warnings.warn(
# solution pip install orjson
# pip install ujson

# No module named 'lightning'
# solution: pip install "gluonts[torch]"


### Get data from SQL


<!-- Get applicant data from DB -->




In [7]:
# import sql ingestion script 
# Data to read from csv so we dont have random data each time
# data = pd.read_csv("sample_account_transactions.csv")
# insert_applicant_fin_history(data, "fin_history")
# Note update data to have unique id for each applicant and no update on conflict with id and date

In [8]:
data = retrieve_data_from_sql("fin_history")
applicant_id = '123456799'
required_amount = 14000  # Example threshold amount



Connected to database...'cwb-database'
Your data has retrieved successfully from the SQL database...
Your connection is closed


In [9]:
# data.head()

### Feature Engineering


### Prepare training data


In [10]:
# Step 2: Prepare data for DeepAR - 
# - Split data 
# - Prep dynamic features
# - normalise where neccessary, 
# - scale balance with appropriate scaler 
# - convert to expected GluonTS format

# Split data: use first 7 months for training, last month for validation
split_idx = len(data) - 30  # Last 30 days as validation
train_data = data.iloc[:split_idx]

training_data, scaler = prep_data_for_deep_ar_model(train_data)


### DeepAR Model building and training


In [11]:

forecasting_model_for_validation = create_model_and_train(training_data)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type        | Params | Mode  | In sizes                                                           | Out sizes   
----------------------------------------------------------------------------------------------------------------------------------
0 | model | DeepARModel | 14.0 K | train | [[1, 1], [1, 1], [1, 1152, 12], [1, 1152], [1, 1152], [1, 30, 12]] | [1, 100, 30]
----------------------------------------------------------------------------------------------------------------------------------
14.0 K    Trainable params
0         Non-trainable params
14.0 K    Total params
0.056     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode


Epoch 0: |          | 12/? [00:01<00:00, 10.23it/s, v_num=0, train_loss=0.819]

Epoch 0, global step 12: 'train_loss' reached 0.81890 (best 0.81890), saving model to '/Users/admin/PycharmProjects/cwb-ml-api/lightning_logs/version_0/checkpoints/epoch=0-step=12.ckpt' as top 1


Epoch 1: |          | 12/? [00:00<00:00, 17.82it/s, v_num=0, train_loss=0.536]

Epoch 1, global step 24: 'train_loss' reached 0.53611 (best 0.53611), saving model to '/Users/admin/PycharmProjects/cwb-ml-api/lightning_logs/version_0/checkpoints/epoch=1-step=24.ckpt' as top 1


...

Epoch 8: |          | 12/? [00:00<00:00, 23.45it/s, v_num=0, train_loss=-1.32]

Epoch 8, global step 108: 'train_loss' reached -1.32394 (best -1.32394), saving model to '/Users/admin/PycharmProjects/cwb-ml-api/lightning_logs/version_0/checkpoints/epoch=8-step=108.ckpt' as top 1


Epoch 9: |          | 12/? [00:00<00:00, 18.54it/s, v_num=0, train_loss=-1.52]

Epoch 9, global step 120: 'train_loss' reached -1.52285 (best -1.52285), saving model to '/Users/admin/PycharmProjects/cwb-ml-api/lightning_logs/version_0/checkpoints/epoch=9-step=120.ckpt' as top 1
`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: |          | 12/? [00:00<00:00, 17.89it/s, v_num=0, train_loss=-1.52]


### Generate forecasts for validation


In [12]:
# Step 4.1: Generate forecasts



validation_forecasts, validation_tss = generate_forecasts(forecasting_model_for_validation, training_data)



In [13]:
# #  Test - not needed for final script in API 
# # We need to ensure the scaler is properly fitted
# # Let's refit it on the original training data
# from sklearn.preprocessing import RobustScaler

# # Get original unscaled values

# # This assumes you have the original data before scaling
# original_values = train_data['balance'].values.reshape(-1, 1)

# # Initialize and fit a new scaler
# scaler = RobustScaler()
# scaler.fit(original_values)

# # Transform a sample to verify the scaler works as expected
# sample_scaled = scaler.transform(original_values[:5])
# sample_restored = scaler.inverse_transform(sample_scaled)
# print("\nScaling verification:")
# print("Original:", original_values[:5].flatten())
# print("\n")
# print("Scaled:", sample_scaled.flatten())
# print("Restored:", sample_restored.flatten())

In [14]:
# Now transform the forecasts
# Step 4.2: Generate forecasts - Transform forecasts back to original scale

transformed_validation_forecast_values = inverse_transform_forecasts(validation_forecasts[0], scaler)

# # Check the transformed values
# print("\nTransformed forecast values:")
# print("p50 (first 5):", transformed_validation_forecast_values['p50'][:5])

In [15]:
(forecast_7days_validation_set, 
 forecast_14days_validation_set, 
 forecast_30days_validation_set
 ) = get_forecast_data_frames(transformed_validation_forecast_values, data)

In [16]:
# forecast_30days_validation_set.head()

# Model validation & evaluation 


In [17]:
# selected_metrics = get_evaluation_metrics(validation_tss, validation_forecasts, scaler)

In [18]:
# selected_metrics

In [19]:
# Calculate RMSE

combined_rmse_df = get_combined_rmse(forecast_7days_validation_set, forecast_14days_validation_set, forecast_30days_validation_set)


# combined_rmse_df


###  save model/ experiment results

In [20]:


# Define the path to the lightning_logs directory
logs_dir = "lightning_logs"



experiment_no = get_experiment_number(logs_dir)
# experiment_no

Highest experiment number found: 0


In [21]:
#  extract hyperparameters




hyperparameters_path = f'lightning_logs/version_{experiment_no}/hparams.yaml'
experiment_id = f'exp_{experiment_no}'
hyperparameters_df = get_hyperparameters(hyperparameters_path, experiment_id)

# print(df)
# print(df)
# hyperparameters_df.head(30)


In [22]:
# Step 6: Extract key metrics for assessment using transformed values


# Final balance predictions (use the last value of each transformed forecast array)
final_median = transformed_validation_forecast_values['p50'][-1]
final_p10 = transformed_validation_forecast_values['p10'][-1]
final_p90 = transformed_validation_forecast_values['p90'][-1]


# Extract actual final balance if available
if len(data) > len(train_data):
    actual_final = data['balance'].iloc[-1]
    error = actual_final - final_p90
    
    # Check if actual falls within the prediction interval
    within_interval = final_p10 <= actual_final <= final_p90
  

# Example assessment for a required amount

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

affordability_assessment = assess_affordability(required_amount, final_p10, final_p90)



In [23]:
# Overall assessment 

experiment_id = f'exp_{experiment_no}'

overall_validation_forecast_assessment_df = get_overall_assessment(
    experiment_id,
    required_amount, 
    affordability_assessment, 
    train_data, 
    final_p10, 
    final_median, 
    final_p90, 
    actual_final,
    error, 
    within_interval,
    )

# overall_validation_forecast_assessment_df
# the one for the test set/future forecst will not have comparative metrics
   

In [24]:
#  Combine overall results and hyperparametres into a data frame

hyperparameters_and_overall_validation_assessment_df = pd.concat([hyperparameters_df, overall_validation_forecast_assessment_df], ignore_index=True)

# Save to csv
# hyperparameters_and_overall_validation_assessment_df.to_csv(f'result_exp_{experiment_no}.csv', index=False) 

In [25]:

# hyperparameters_and_overall_validation_assessment_df.head(50)

In [26]:
# insert in database 
# ................MODIFY TO HAVE UNIQUE ID FOR EACH APPLICANT................
#  1. Forecasts x 30 days 
#  2. feature engineered data (actual balance including 7days rolling average)
#  3. Converted to gbp
#  3. Hyperparameters & overall validation assessment

# Database tables 

dbname='cwb-database',

#  delete/DROP later
'cwb_results' # old table for holding results

##### Active tables
# Financial history 
fin_history_table_name = 'fin_history'
fin_history_enhanced_table_name = 'fin_history_enhanced' # Feature engineered data, applicant id, date, time etc

# Applicant Assessment and Model Evaluation 
cwb_combined_rmse_table_name = 'cwb_combined_rmse' # Combined results for validation and future set
cwb_validation_assessment_table_name = 'cwb_validation_assessment' # Assessment and Hyperparameters for validation set
# cwb_future_assessment_table_name = 'cwb_future_assessment' # Assessment and Hyperparameters for future set

# Forecasts
cwb_validation_forecasts_table_name = 'cwb_validation_forecasts' # 30 days forecast, date, actual balance
# gbp_cwb_validation_forecasts_table_name = 'gbp_cwb_validation_forecasts' # 30 days forecast, date, actual balance
# cwb_future_forecasts_table_name = 'cwb_future_forecasts'  # 30 days forecast, date
# gbp_cwb_future_forecasts_table_name = 'gbp_cwb_future_forecasts'  # 30 days forecast, date






In [27]:
# Insert into database

# Add meta data to Financial history enhanced
data_df = add_metadata_columns(data, applicant_id = applicant_id)

# Add metadata columns to combined RMSE
combined_rmse_df = add_metadata_columns(combined_rmse_df, applicant_id = applicant_id)

# Add meta data to relevant dataframes 
hyperparameters_and_overall_validation_assessment_df = add_metadata_columns(hyperparameters_and_overall_validation_assessment_df, applicant_id = applicant_id)

# Add metadata columns to validation forecasts
forecast_30days_validation_set_df = add_metadata_columns(forecast_30days_validation_set, applicant_id = applicant_id)


# 1. Financial history enhanced
cwb_fin_history_enhanced_dict = get_column_name_and_datatype_dictionary(data_df)
cwb_fin_history_enhanced_sql_queries_and_values = prepare_sql_queries_and_values(cwb_fin_history_enhanced_dict, fin_history_enhanced_table_name, data_df)
insert_data_into_sql_data_base(*cwb_fin_history_enhanced_sql_queries_and_values)

# 2. Combined RMSE
cwb_rmse_dict = get_column_name_and_datatype_dictionary(combined_rmse_df)
cwb_rsme_sql_queries_and_values = prepare_sql_queries_and_values(cwb_rmse_dict, cwb_combined_rmse_table_name, combined_rmse_df)
insert_data_into_sql_data_base(*cwb_rsme_sql_queries_and_values)

# 3. Validation Assessment Results (and Hyperparameters)
cwb_validation_results_dict = get_column_name_and_datatype_dictionary(hyperparameters_and_overall_validation_assessment_df)
cwb_validation_results_sql_queries_and_values = prepare_sql_queries_and_values(cwb_validation_results_dict, cwb_validation_assessment_table_name, hyperparameters_and_overall_validation_assessment_df)
insert_data_into_sql_data_base(*cwb_validation_results_sql_queries_and_values)

# 4. Validation Forecasts
cwb_validation_forecasts_dict = get_column_name_and_datatype_dictionary(forecast_30days_validation_set_df)
cwb_validation_forecasts_sql_queries_and_values = prepare_sql_queries_and_values(cwb_validation_forecasts_dict, cwb_validation_forecasts_table_name, forecast_30days_validation_set_df)
insert_data_into_sql_data_base(*cwb_validation_forecasts_sql_queries_and_values)




Connected to database...'cwb-database'
Inserting 240 rows in a batch...into table 'fin_history_enhanced'
Your data has been inserted successfully into the SQL database...
Your connection is closed


Connected to database...'cwb-database'
Inserting 14 rows in a batch...into table 'cwb_combined_rmse'
Your data has been inserted successfully into the SQL database...
Your connection is closed


Connected to database...'cwb-database'
Inserting 33 rows in a batch...into table 'cwb_validation_assessment'
Your data has been inserted successfully into the SQL database...
Your connection is closed


Connected to database...'cwb-database'
Inserting 30 rows in a batch...into table 'cwb_validation_forecasts'
Your data has been inserted successfully into the SQL database...
Your connection is closed



In [32]:
retrieve_data_from_sql(cwb_validation_assessment_table_name)


Connected to database...'cwb-database'
Your data has retrieved successfully from the SQL database...
Your connection is closed


,category,metric,value,experimentid,applicant_id,date_added,transaction_time,sn
0,Hyperparameter,lr,0.001,exp_1,123456799,2025-03-28,2025-03-28 23:46:17.483446,1
1,Hyperparameter,model_kwargs.cardinality.context_length,60,exp_1,123456799,2025-03-28,2025-03-28 23:46:17.483446,2
2,Hyperparameter,model_kwargs.cardinality.default_scale,None,exp_1,123456799,2025-03-28,2025-03-28 23:46:17.483446,3
3,Hyperparameter,state.__init_args__,*id001,exp_1,123456799,2025-03-28,2025-03-28 23:46:17.483446,4
4,Hyperparameter,state.beta,0.0,exp_1,123456799,2025-03-28,2025-03-28 23:46:17.483446,5
5,Hyperparameter,dropout_rate,0.1,exp_1,123456799,2025-03-28,2025-03-28 23:46:17.483446,6
6,Hyperparameter,embedding_dimension,None,exp_1,123456799,2025-03-28,2025-03-28 23:46:17.483446,7
7,Hyperparameter,freq,D,exp_1,123456799,2025-03-28,2025-03-28 23:46:17.483446,8
8,Hyperparameter,hidden_size,40,exp_1,123456799,2025-03-28,2025-03-28 23:46:17.483446,9
9,Hyperparameter,lags_seq,None,exp_1,123456799,2025-03-28,2025-03-28 23:46:17.483446,10


In [29]:
retrieve_data_from_sql(cwb_validation_assessment_table_name)


Connected to database...'cwb-database'
Your data has retrieved successfully from the SQL database...
Your connection is closed


,category,metric,value,experimentid,applicant_id,date_added,transaction_time,sn
0,Hyperparameter,lr,0.001,exp_0,123456799,2025-03-28,2025-03-28 23:31:40.399818,1
1,Hyperparameter,model_kwargs.cardinality.context_length,60,exp_0,123456799,2025-03-28,2025-03-28 23:31:40.399818,2
2,Hyperparameter,model_kwargs.cardinality.default_scale,None,exp_0,123456799,2025-03-28,2025-03-28 23:31:40.399818,3
3,Hyperparameter,state.__init_args__,*id001,exp_0,123456799,2025-03-28,2025-03-28 23:31:40.399818,4
4,Hyperparameter,state.beta,0.0,exp_0,123456799,2025-03-28,2025-03-28 23:31:40.399818,5
5,Hyperparameter,dropout_rate,0.1,exp_0,123456799,2025-03-28,2025-03-28 23:31:40.399818,6
6,Hyperparameter,embedding_dimension,None,exp_0,123456799,2025-03-28,2025-03-28 23:31:40.399818,7
7,Hyperparameter,freq,D,exp_0,123456799,2025-03-28,2025-03-28 23:31:40.399818,8
8,Hyperparameter,hidden_size,40,exp_0,123456799,2025-03-28,2025-03-28 23:31:40.399818,9
9,Hyperparameter,lags_seq,None,exp_0,123456799,2025-03-28,2025-03-28 23:31:40.399818,10


In [33]:
retrieve_data_from_sql(cwb_combined_rmse_table_name)


Connected to database...'cwb-database'
Your data has retrieved successfully from the SQL database...
Your connection is closed


,quartiles,seven_day_forecast,fourteen_day_forecast,thirty_day_forecast,applicant_id,date_added,transaction_time,sn
0,p60,2208.581398,2305.149604,2330.719699,123456799,2025-03-28,2025-03-28 23:46:17.481512,8
1,p70,1915.740455,2029.371308,1991.747160,123456799,2025-03-28,2025-03-28 23:46:17.481512,9
2,p80,1563.117432,1702.546614,1588.419685,123456799,2025-03-28,2025-03-28 23:46:17.481512,10
3,p90,956.931942,1137.065585,1055.090606,123456799,2025-03-28,2025-03-28 23:46:17.481512,11
4,p92,778.986692,956.558753,955.390617,123456799,2025-03-28,2025-03-28 23:46:17.481512,12
5,p95,369.701235,576.882882,961.641766,123456799,2025-03-28,2025-03-28 23:46:17.481512,13
6,p99,1974.845226,1540.474288,3366.727046,123456799,2025-03-28,2025-03-28 23:46:17.481512,14
7,p1,6423.114584,6439.478992,7893.917531,123456799,2025-03-28,2025-03-28 23:46:17.481512,1
8,p5,4529.454391,4546.161912,5278.280460,123456799,2025-03-28,2025-03-28 23:46:17.481512,2
9,p10,3884.451077,3941.014227,4450.001300,123456799,2025-03-28,2025-03-28 23:46:17.481512,3


In [31]:
retrieve_data_from_sql(cwb_validation_forecasts_table_name).tail(14)


Connected to database...'cwb-database'
Your data has retrieved successfully from the SQL database...
Your connection is closed


,p1,p5,p10,p20,p30,p40,p50,p60,p70,p80,p90,p92,p95,p99,actual,date,applicant_id,date_added,transaction_time,sn
16,9907.409180,11036.418945,11452.928711,11901.124023,12167.172852,12382.263672,12582.376953,12757.944336,12957.586914,13244.069336,13662.960938,13736.888672,14103.295898,15502.991211,13625.501662,2024-08-14,123456799,2025-03-28,2025-03-28 23:31:40.401446,17
17,9650.589844,10842.444336,11301.734375,11753.022461,12047.130859,12312.342773,12498.442383,12681.937500,12915.221680,13132.199219,13506.544922,13604.148438,13870.355469,14605.666992,13563.539592,2024-08-15,123456799,2025-03-28,2025-03-28 23:31:40.401446,18
18,9957.764648,10926.379883,11347.509766,11717.827148,12006.525391,12225.372070,12435.699219,12624.218750,12917.220703,13144.570312,13527.382812,13634.591797,13917.482422,15182.388672,13536.759244,2024-08-16,123456799,2025-03-28,2025-03-28 23:31:40.401446,19
19,9292.201172,10654.292969,11001.945312,11471.726562,11727.978516,11984.978516,12209.434570,12399.547852,12620.777344,12936.227539,13381.623047,13516.767578,13917.806641,15009.755859,13407.184192,2024-08-17,123456799,2025-03-28,2025-03-28 23:31:40.401446,20
20,9550.059570,10455.055664,10872.625977,11367.304688,11702.017578,11945.208008,12179.417969,12405.591797,12673.821289,12995.672852,13436.546875,13588.818359,14029.011719,15407.141602,13378.803278,2024-08-18,123456799,2025-03-28,2025-03-28 23:31:40.401446,21
21,9208.435547,10188.960938,10628.969727,11227.491211,11577.980469,11905.070312,12142.081055,12373.455078,12622.317383,12912.004883,13439.855469,13556.098633,13888.018555,15409.569336,13367.922521,2024-08-19,123456799,2025-03-28,2025-03-28 23:31:40.401446,22
22,8554.883789,10013.317383,10665.636719,11263.921875,11633.582031,11953.981445,12239.922852,12501.636719,12761.730469,13105.328125,13659.255859,13814.502930,14185.467773,15690.592773,13362.229295,2024-08-20,123456799,2025-03-28,2025-03-28 23:31:40.401446,23
23,8598.462891,9943.643555,10534.176758,11247.896484,11604.220703,11927.288086,12186.363281,12463.806641,12744.906250,13145.830078,13744.800781,13841.085938,14251.097656,15564.326172,13353.429012,2024-08-21,123456799,2025-03-28,2025-03-28 23:31:40.401446,24
24,7890.364746,9967.493164,10554.040039,11207.917969,11670.823242,12015.736328,12313.414062,12590.650391,12932.054688,13275.544922,13815.708984,13977.695312,14411.205078,15665.235352,13320.715951,2024-08-22,123456799,2025-03-28,2025-03-28 23:31:40.401446,25
25,8223.371094,9850.517578,10536.138672,11140.032227,11570.545898,11904.000977,12209.121094,12547.916016,12885.477539,13340.058594,14002.144531,14204.565430,14821.710938,16871.960938,13318.248344,2024-08-23,123456799,2025-03-28,2025-03-28 23:31:40.401446,26
